<a href="https://colab.research.google.com/github/mrobotov/HW/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Song classification using Variational Auto-Encoder**


# First note

first thing, copy the following files from the sample data folder up:
DataSet.py
variational_auto_encoder.py

dont run this before you read the instructions on the bottom



# Code
starting with imports:

In [10]:
! pip install python_speech_features
from DataSet import DataSet
from scipy.io import wavfile
import python_speech_features as psf
import numpy as np
from sklearn.preprocessing import StandardScaler
from variational_auto_encoder import Vae, beta_loss_function
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import time
transform = torchvision.transforms.ToTensor()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


this piece of code defines the function we will use to perform the window in the MFCC,
the function we use to read the files and the mfcc transform


In [8]:
mfcc_winfunc = lambda x: np.blackman(x)  # np.hamming(x) np.kaiser(M=x, beta=14)# # 'Hamming' # [ frame window type ]
def file_read(filename):
    fs,data = wavfile.read(filename)
    return fs,data


def extract_features(data,params):
    X_mfcc = psf.mfcc(  # The audio signal from which to compute mfcc_features.
        data.transpose(),
        # The samplerate of the signal we are working with.
        params.fs,
        # The length of the analysis window in seconds.
        # Default is 0.025s (25 milliseconds)
        winlen=params.mfcc_winsize,
        # The step between successive windows in seconds.
        # Default is 0.01s (10 milliseconds)
        winstep=params.mfcc_winstep,
        # The number of cepstrum to return.
        # Default 13.
        numcep=params.mfcc_num_coeffs,
        # The number of filters in the filterbank.
        # Default is 26.
        nfilt=params.mfcc_filters,
        # The FFT size. Default is 512.
        nfft=params.mfcc_nfft,
        # If true, the zeroth cepstral coefficient is replaced
        # with the log of the total frame energy.
        appendEnergy=True,
        # use this window for framing
        winfunc=mfcc_winfunc)
    return np.square(np.abs(np.fft.fft(X_mfcc)))











Here we do the file read
NOTICE - the name of the directory here is: 'debug set'
you need to change it when you load your own dir

this directory should contain dub directories with the genre/class name and the songs inside each sub dir

(length doesn't matteer right now)

In [11]:
set_name='debug set'
data_set = DataSet(name=set_name,debug_mode=True,file_read=file_read)
data_set.read()


Set - Initialized Set debug set


FileNotFoundError: ignored

and transforming the files to a matrix (or rather a tensor, since we are using pytorch):

starting with the case that we want to perform MFCC:


In [ ]:
class parameters:
    def __init__(self,mfcc_winsize=0.02,mfcc_winstep=0.01,mfcc_num_coeffs=13,mfcc_filters=26,mfcc_nfft=int(32768 / 32),kmeans_cluster=15):
        self.mfcc_winsize =  mfcc_winsize # [seconds] - affects M dimension of feature vector
        self.mfcc_winstep = mfcc_winstep  # [seconds] - affects feature vector smoothness
        self.mfcc_num_coeffs = mfcc_num_coeffs  # [number of Mel Frequency Cepstrum Coefficient] feature vector N dimension
        self.mfcc_filters = mfcc_filters # [number of MFCC filters] has to be larger or equal to NUM_MFCC
        self.mfcc_nfft = mfcc_nfft  # [ number of frames for fft]
        self.kmeans_cluster = kmeans_cluster  # [ number of kmeans cluster for features ]
        self.mfcc_winfunc = mfcc_winfunc
        self.fs = data_set.fs
        
for x_vec,y_class in zip(data_set.X,data_set.y):

    X_mfcc = extract_features(x_vec,params)
    if len(X) == 0:
        X = X_mfcc
        y = np.repeat(y_class,np.size(X_mfcc,0))
    else:
        X = np.vstack((X, X_mfcc))
        y = np.append(y,np.repeat(y_class,np.size(X_mfcc,0)))

In [ ]:
X = []
y = []
X = data_set.X
y = data_set.y







### **OK now we start with the fun**

the following params are:

FRAME_SIZE - number of dimensions per sample, in case we take the raw audio this should be Fs* [Number of seconds] (44100*N)

Z_DIM - the number of dimensions the algorithm will try to represent the input 

> data (in the example of the digits it was Z = 10 )

BETA - this is related to this specific version of the algorithm 
this beta scales between two errors, in the form of:

> E_total = E_1 + beta*E_2
> the range is about ~ 0.05 - 10
> when 1 is considered the normal "Vanilla" version

NUM_EPOCHS - an epoch is a full pass over the training set.
> when we train we want to pass a lots of times on the set

LEARNING_RATE - this impacts the learning step size.
> if the error from epoch to epoch is overshooting, we should conider reducing the leanrning rate

HIDDEN_SIZE - this parameter is related to the algorithm itself and it contrpls the size of the intermidaite level of neurons


following the parameters is scaling of X
and some trial of capturing a pytorch remote machine the has a gpu

after that some required inits:

In [9]:
FRAME_SIZE = int(44100/60)
Z_DIM = 10 # size of the latent dimension
BETA =  0.05 #[0.05,0.5,1,5]
LEARNING_RATE = 1e-4 # for the gradient optimizer
NUM_EPOCHS = 100 #
HIDDEN_SIZE =  1024 # size of the hidden layers in the networks


X = X.reshape(-1,X_DIM)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# check if there is gpu avilable, if there is, use it
if torch.cuda.is_available():
    torch.cuda.current_device()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print("running calculations on: ", device)
# create our model and send it to the device (cpu/gpu)
vae = Vae(x_dim=X_DIM, z_dim=Z_DIM, hidden_size=HIDDEN_SIZE, device=device).to(device)
# optimizer
vae_optim = torch.optim.Adam(params=vae.parameters(), lr=LEARNING_RATE)
x = transform(X_scaled)#.to(device).view(-1, X_DIM)  # just the images
train_losses = []
recon_mean = []
kl_mean = []















## here the learning begins:


In [ ]:
for epoch in range(NUM_EPOCHS):
    epoch_start_time = time.time()
    batch_losses = []
    recon_loss = []
    kl_loss = []

    x_recon, mu, logvar, z = vae(x.to(device=device, dtype=torch.float32))
    # calculate the loss
    recon, kl, loss = beta_loss_function(x_recon, x.to(device=device, dtype=torch.float32), mu, logvar, loss_type='bce', beta=BETA)
    # optimization (same 3 steps everytime)
    vae_optim.zero_grad()
    loss.backward()
    vae_optim.step()
    # save loss
    batch_losses.append(loss.data.cpu().item())
    recon_loss.append(recon)
    kl_loss.append(kl)

    train_losses.append(np.mean(batch_losses))
    if epoch > 0:
        print("epoch: {} training loss: {:.3f} stride: {:.2f} epoch time: {:.2f} sec".format(epoch, train_losses[-1], train_losses[-2]-train_losses[-1],
                                                                          time.time() - epoch_start_time))
    recon_mean.append(np.array(recon_loss).mean())
    kl_mean.append(np.array(kl_loss).mean())
    # save
fname = "beta_(" + str(BETA) + " )_vae.pth"
torch.save(vae.state_dict(), fname)
print("saved checkpoint @", fname)

at the end it saves the vae dict to a file which can be loaded with:

In [ ]:
# load
vae=Vae(x_dim=X_DIM,z_dim=Z_DIM,hidden_size=HIDDEN_SIZE,device=device).to(device)vae.load_state_dict(torch.load(fname))
print("loaded checkpoint from",fname)

# Instructions

(1) connect this code with as large as possible song data base

(2) go through tthe notebook, and run section by section

(3) create a loop that runs on different variations of params, somting like this:

In [ ]:
BETA =  [0.05,0.5,1,5]
for beta in BETA:
  # do stuff


(4) write a function that converts the matrix back to an audio file
> test it with the original X

(5) try to find the params that best reconstruct the audio

key notes:
> enlarging the Z dim can improve results due to richer representation
> learning_rate might be changeable from epoch to epoch - explore it
> the larger the frame size the chalenging it is to forecast it
> larger hidden layer size larger pattern recognition ability - but beware of overfitting - that is when we learn the particular data set "too" good

good luck!!  